In [ ]:
! pip install geopandas
! pip install pandas

In [1]:
import geopandas as gpd
import pandas as pd

In [5]:
SHAPE_PATH = "/content/lct_000b21a_e.shp"
gdf = gpd.read_file(SHAPE_PATH)
gdf = gdf.to_crs("EPSG:4326")
gdf.head()

,CTUID,DGUID,CTNAME,LANDAREA,PRUID,geometry
0,5370001.08,2021S05075370001.08,0001.08,1.6383,35,"POLYGON ((-79.85362 43.1932, -79.8538 43.19277..."
1,0010002.00,2021S05070010002.00,0002.00,1.9638,10,"POLYGON ((-52.7205 47.55154, -52.71877 47.5507..."
2,5370001.09,2021S05075370001.09,0001.09,1.9699,35,"POLYGON ((-79.85586 43.1879, -79.85592 43.1877..."
3,5370120.02,2021S05075370120.02,0120.02,76.9650,35,"POLYGON ((-79.94562 43.1692, -79.94637 43.1675..."
4,0010006.00,2021S05070010006.00,0006.00,1.0467,10,"POLYGON ((-52.71107 47.56251, -52.71143 47.562..."


In [15]:
toronto_gdf = gdf[gdf['CTUID'].str.startswith('5350')]
toronto_gdf = toronto_gdf[toronto_gdf['CTUID'].between('5350001', '5350400')]
# toronto_gdf.head()

toronto_gdf_boundaries = toronto_gdf.copy()
toronto_gdf['centroid'] = toronto_gdf.geometry.centroid
trt_centroids_gdf = gpd.GeoDataFrame(toronto_gdf[['CTUID']], geometry=toronto_gdf['centroid'])
trt_centroids_gdf.head()

<ipython-input-15-45390a5e04d8>:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  toronto_gdf['centroid'] = toronto_gdf.geometry.centroid


,CTUID,geometry
486,5350128.04,POINT (-79.39144 43.69746)
501,5350363.06,POINT (-79.24815 43.77771)
505,5350363.07,POINT (-79.23027 43.78152)
507,5350378.23,POINT (-79.26838 43.79914)
508,5350378.24,POINT (-79.25399 43.80412)


In [11]:
geojson_file = "/content/Neighbourhoods - 4326.geojson"
neighbourhood = gpd.read_file(geojson_file)
neighbourhood = neighbourhood.to_crs("EPSG:4326")
neighbourhood.head()

,_id,AREA_ID,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_NAME,AREA_DESC,CLASSIFICATION,CLASSIFICATION_CODE,OBJECTID,geometry
0,1,2502366,26022881,0,174,174,South Eglinton-Davisville,South Eglinton-Davisville (174),Not an NIA or Emerging Neighbourhood,NA,17824737.0,"MULTIPOLYGON (((-79.38635 43.69783, -79.38623 ..."
1,2,2502365,26022880,0,173,173,North Toronto,North Toronto (173),Not an NIA or Emerging Neighbourhood,NA,17824753.0,"MULTIPOLYGON (((-79.39744 43.70693, -79.39837 ..."
2,3,2502364,26022879,0,172,172,Dovercourt Village,Dovercourt Village (172),Not an NIA or Emerging Neighbourhood,NA,17824769.0,"MULTIPOLYGON (((-79.43411 43.66015, -79.43537 ..."
3,4,2502363,26022878,0,171,171,Junction-Wallace Emerson,Junction-Wallace Emerson (171),Not an NIA or Emerging Neighbourhood,NA,17824785.0,"MULTIPOLYGON (((-79.4387 43.66766, -79.43841 4..."
4,5,2502362,26022877,0,170,170,Yonge-Bay Corridor,Yonge-Bay Corridor (170),Not an NIA or Emerging Neighbourhood,NA,17824801.0,"MULTIPOLYGON (((-79.38404 43.64497, -79.38502 ..."


In [16]:
join_ct_w_n = combined = gpd.sjoin(trt_centroids_gdf, neighbourhood, how="left", predicate="within")

In [17]:
join_ct_w_n.head()

,CTUID,geometry,index_right,_id,AREA_ID,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_NAME,AREA_DESC,CLASSIFICATION,CLASSIFICATION_CODE,OBJECTID
486,5350128.04,POINT (-79.39144 43.69746),0.0,1.0,2502366.0,26022881.0,0.0,174,174,South Eglinton-Davisville,South Eglinton-Davisville (174),Not an NIA or Emerging Neighbourhood,NA,17824737.0
501,5350363.06,POINT (-79.24815 43.77771),20.0,21.0,2502346.0,26022861.0,0.0,142,142,Woburn North,Woburn North (142),Neighbourhood Improvement Area,NIA,17825057.0
505,5350363.07,POINT (-79.23027 43.78152),20.0,21.0,2502346.0,26022861.0,0.0,142,142,Woburn North,Woburn North (142),Neighbourhood Improvement Area,NIA,17825057.0
507,5350378.23,POINT (-79.26838 43.79914),30.0,31.0,2502336.0,26022851.0,0.0,129,129,Agincourt North,Agincourt North (129),Not an NIA or Emerging Neighbourhood,NA,17825217.0
508,5350378.24,POINT (-79.25399 43.80412),30.0,31.0,2502336.0,26022851.0,0.0,129,129,Agincourt North,Agincourt North (129),Not an NIA or Emerging Neighbourhood,NA,17825217.0


In [18]:
census_tract_n = join_ct_w_n[['CTUID', 'AREA_NAME']]
census_tract_n.head()

,CTUID,AREA_NAME
486,5350128.04,South Eglinton-Davisville
501,5350363.06,Woburn North
505,5350363.07,Woburn North
507,5350378.23,Agincourt North
508,5350378.24,Agincourt North


In [31]:
census_tract_n[census_tract_n['AREA_NAME'].isnull()]

,CTUID,AREA_NAME
906,5350003.00,NaN
3197,5350200.01,NaN
5860,5350210.04,NaN


In [38]:
census_tract_n[census_tract_n['CTUID']=="5350003.00"]['AREA_NAME']


<ipython-input-38-5c5e30cd2e54>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  census_tract_n[census_tract_n['CTUID']=="5350003.00"]['AREA_NAME'] = "South Parkdale"


In [44]:
census_tract_n[census_tract_n['CTUID']=="5350200.01"]

,CTUID,AREA_NAME
3197,5350200.01,NaN


In [41]:
census_tract_n.loc[906, "AREA_NAME"] = "South Parkdale"

<ipython-input-41-31b35653198c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  census_tract_n.loc[906, "AREA_NAME"] = "South Parkdale"


In [45]:
census_tract_n.loc[3197, "AREA_NAME"] = "Humber Bay Shores"

<ipython-input-45-f9da23fcbc06>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  census_tract_n.loc[3197, "AREA_NAME"] = "Humber Bay Shores"


In [46]:
census_tract_n.loc[5860, "AREA_NAME"] = "Humber Bay Shores"

<ipython-input-46-bea3932a9abc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  census_tract_n.loc[5860, "AREA_NAME"] = "Humber Bay Shores"


In [47]:
census_tract_n[census_tract_n['AREA_NAME'].isnull()]

,CTUID,AREA_NAME


In [48]:
census_tract_n.groupby('AREA_NAME').count()

,CTUID
AREA_NAME,
Agincourt North,6
Agincourt South-Malvern West,5
Alderwood,2
Annex,6
Avondale,3
...,...
Yonge-Doris,4
Yonge-Eglinton,2
Yonge-St.Clair,4


In [49]:
census_tract_n.to_csv("CTUID-w-Neighborhood.csv", index=False)